[Reference](https://levelup.gitconnected.com/llms-meet-sql-revolutionizing-data-querying-with-natural-language-processing-52487337f043)

# Part: 1 — Choose the SQL Database, Create the Schema, and Load the data

```
CREATE DATABASE SalesOrderSchema;

USE SalesOrderSchema;

CREATE TABLE Customer (
  CustomerID INT AUTO_INCREMENT PRIMARY KEY,
  FirstName VARCHAR(100),
  LastName VARCHAR(100),
  Email VARCHAR(255),
  Phone VARCHAR(20),
  BillingAddress TEXT,
  ShippingAddress TEXT,
  CustomerSince DATE,
  IsActive BOOLEAN
);

CREATE TABLE SalesOrder (
  SalesOrderID INT AUTO_INCREMENT PRIMARY KEY,
  CustomerID INT,
  OrderDate DATE,
  RequiredDate DATE,
  ShippedDate DATE,
  Status VARCHAR(50),
  Comments TEXT,
  PaymentMethod VARCHAR(50),
  IsPaid BOOLEAN,
  FOREIGN KEY (CustomerID) REFERENCES Customer(CustomerID)
);

CREATE TABLE Product (
  ProductID INT AUTO_INCREMENT PRIMARY KEY,
  ProductName VARCHAR(255),
  Description TEXT,
  UnitPrice DECIMAL(10, 2),
  StockQuantity INT,
  ReorderLevel INT,
  Discontinued BOOLEAN
);

CREATE TABLE LineItem (
  LineItemID INT AUTO_INCREMENT PRIMARY KEY,
  SalesOrderID INT,
  ProductID INT,
  Quantity INT,
  UnitPrice DECIMAL(10, 2),
  Discount DECIMAL(10, 2),
  TotalPrice DECIMAL(10, 2),
  FOREIGN KEY (SalesOrderID) REFERENCES SalesOrder(SalesOrderID),
  FOREIGN KEY (ProductID) REFERENCES Product(ProductID)
);

CREATE TABLE Employee (
  EmployeeID INT AUTO_INCREMENT PRIMARY KEY,
  FirstName VARCHAR(100),
  LastName VARCHAR(100),
  Email VARCHAR(255),
  Phone VARCHAR(20),
  HireDate DATE,
  Position VARCHAR(100),
  Salary DECIMAL(10, 2)
);

CREATE TABLE Supplier (
  SupplierID INT AUTO_INCREMENT PRIMARY KEY,
  CompanyName VARCHAR(255),
  ContactName VARCHAR(100),
  ContactTitle VARCHAR(50),
  Address TEXT,
  Phone VARCHAR(20),
  Email VARCHAR(255)
);

CREATE TABLE InventoryLog (
  LogID INT AUTO_INCREMENT PRIMARY KEY,
  ProductID INT,
  ChangeDate DATE,
  QuantityChange INT,
  Notes TEXT,
  FOREIGN KEY (ProductID) REFERENCES Product(ProductID)
);
```

```
SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE, IS_NULLABLE, COLUMN_DEFAULT, CHARACTER_MAXIMUM_LENGTH
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_SCHEMA = 'SalesOrderSchema'
ORDER BY TABLE_NAME, ORDINAL_POSITION;
```

In [1]:
#The code for loading data into the customer table
#Customer Table
import mysql.connector
from faker import Faker

# Initialize Faker
fake = Faker()

# Connect to MySQL
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Your MySQL Password",
  database="SalesOrderSchema"
)
cursor = conn.cursor()

# Generate and insert data
for _ in range(100):  # Let's say we want to generate 100 records
    first_name = fake.first_name()
    last_name = fake.last_name()
    email = fake.email()
    phone = fake.phone_number()
    if len(phone) > 20:  # Assuming the 'Phone' column is VARCHAR(20)
        phone = phone[:20]  # Truncate phone number to fit into the column
    address = fake.address()
    customer_since = fake.date_between(start_date='-5y', end_date='today')
    is_active = fake.boolean()

    # Insert customer data
    cursor.execute("""
        INSERT INTO Customer (FirstName, LastName, Email, Phone, BillingAddress, ShippingAddress, CustomerSince, IsActive)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (first_name, last_name, email, phone, address, address, customer_since, is_active))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

In [2]:
#Employee Table
import mysql.connector
from faker import Faker

# Initialize Faker
fake = Faker()

# Connect to MySQL
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Your MySQL Password",
  database="SalesOrderSchema"
)
cursor = conn.cursor()

# Generate and insert 1000 employee records
for _ in range(1000):
    first_name = fake.first_name()
    last_name = fake.last_name()
    email = fake.email()
    phone = fake.phone_number()
    if len(phone) > 20:  # Truncate phone number if necessary
        phone = phone[:20]
    hire_date = fake.date_between(start_date='-5y', end_date='today')
    position = fake.job()
    salary = round(fake.random_number(digits=5), 2)  # Generate a 5 digit salary

    # Insert employee data
    cursor.execute("""
        INSERT INTO Employee (FirstName, LastName, Email, Phone, HireDate, Position, Salary)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (first_name, last_name, email, phone, hire_date, position, salary))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("1000 employee records inserted successfully.")

In [3]:
#Product Table
import mysql.connector
from faker import Faker
import random

# Initialize Faker
fake = Faker()

# Connect to MySQL
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Your MySQL Password",
  database="SalesOrderSchema"
)
cursor = conn.cursor()

# Generate and insert data into the Product table
for _ in range(1000):  # Generate 1000 product records
    product_name = fake.word().capitalize() + " " + fake.word().capitalize()
    description = fake.sentence(nb_words=10)
    unit_price = round(random.uniform(10, 500), 2)  # Random price between $10 and $500
    stock_quantity = random.randint(10, 1000)  # Random stock quantity between 10 and 1000
    reorder_level = random.randint(5, 50)  # Random reorder level between 5 and 50
    discontinued = random.choice([0, 1])  # Randomly choose between 0 (false) and 1 (true)

    # Insert product data
    cursor.execute("""
        INSERT INTO Product (ProductName, Description, UnitPrice, StockQuantity, ReorderLevel, Discontinued)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (product_name, description, unit_price, stock_quantity, reorder_level, discontinued))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("Products inserted successfully.")
#Supplier Table
import mysql.connector
from faker import Faker
import random

# Initialize Faker
fake = Faker()

# Connect to MySQL
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Your MySQL Password",
  database="SalesOrderSchema"
)
cursor = conn.cursor()

# Generate and insert data into the Supplier table
for _ in range(1000):  # Assuming you want to insert 1000 records
    company_name = fake.company()
    contact_name = fake.name()
    contact_title = fake.job()
    # Ensure ContactTitle does not exceed the column's max length, e.g., VARCHAR(50)
    contact_title = contact_title[:50] if len(contact_title) > 50 else contact_title
    address = fake.address().replace('\n', ', ')  # Replace newlines with commas for address
    phone = fake.phone_number()
    # Ensure phone does not exceed the column's max length, e.g., VARCHAR(20)
    phone = phone[:20] if len(phone) > 20 else phone
    email = fake.email()

    # Insert supplier data
    cursor.execute("""
        INSERT INTO Supplier (CompanyName, ContactName, ContactTitle, Address, Phone, Email)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (company_name, contact_name, contact_title, address, phone, email))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("Suppliers inserted successfully.")

In [4]:
#Sales Order Table
import mysql.connector
from faker import Faker
from datetime import timedelta
import random

fake = Faker()

conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Your MySQL Password",
  database="SalesOrderSchema"
)
cursor = conn.cursor(buffered=True)

# Fetch customer IDs
cursor.execute("SELECT CustomerID FROM Customer")
customer_ids = [id[0] for id in cursor.fetchall()]

# Insert data into SalesOrder
for _ in range(1000):  # Let's say we want to generate 1000 sales orders
    customer_id = random.choice(customer_ids)
    order_date = fake.date_between(start_date='-2y', end_date='today')
    required_date = order_date + timedelta(days=random.randint(1, 30))
    shipped_date = order_date + timedelta(days=random.randint(1, 30)) if random.choice([True, False]) else None
    status = random.choice(['Pending', 'Completed', 'Shipped'])
    is_paid = random.choice([True, False])

    cursor.execute("""
        INSERT INTO SalesOrder (CustomerID, OrderDate, RequiredDate, ShippedDate, Status, IsPaid)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (customer_id, order_date, required_date, shipped_date, status, is_paid))

conn.commit()

In [5]:
#Sales Order Line Item
# Fetch product IDs
cursor.execute("SELECT ProductID FROM Product")
product_ids = [id[0] for id in cursor.fetchall()]

# Fetch sales order IDs
cursor.execute("SELECT SalesOrderID FROM SalesOrder")
sales_order_ids = [id[0] for id in cursor.fetchall()]

# Insert data into LineItem
for _ in range(5000):  # Assuming multiple line items per order
    sales_order_id = random.choice(sales_order_ids)
    product_id = random.choice(product_ids)
    quantity = random.randint(1, 10)
    unit_price = round(random.uniform(10, 100), 2)  # Assuming you have this info or fetch it from Product table
    total_price = quantity * unit_price

    cursor.execute("""
        INSERT INTO LineItem (SalesOrderID, ProductID, Quantity, UnitPrice, TotalPrice)
        VALUES (%s, %s, %s, %s, %s)
    """, (sales_order_id, product_id, quantity, unit_price, total_price))

conn.commit()
cursor.close()
conn.close()

In [6]:
#Inventory Table
import mysql.connector
from faker import Faker
import random

# Initialize Faker
fake = Faker()

# Connect to MySQL
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Your MySQL Password",
  database="SalesOrderSchema"
)
cursor = conn.cursor()

# Fetch Product IDs
cursor.execute("SELECT ProductID FROM Product")
product_ids = [row[0] for row in cursor.fetchall()]

# Assuming you want to insert 1000 inventory log records
for _ in range(1000):
    product_id = random.choice(product_ids)  # Randomly select a product ID
    change_date = fake.date_between(start_date="-1y", end_date="today")
    quantity_change = random.randint(-100, 100)  # Assuming inventory can increase or decrease
    notes = "Inventory " + ("increased" if quantity_change > 0 else "decreased")

    # Insert inventory log data
    cursor.execute("""
        INSERT INTO InventoryLog (ProductID, ChangeDate, QuantityChange, Notes)
        VALUES (%s, %s, %s, %s)
    """, (product_id, change_date, quantity_change, notes))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("Inventory logs inserted successfully.")

# Part: 2:Agents & SQL Agent:

In [7]:
import os
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

# Set your OpenAI API key here
os.environ["OPENAI_API_KEY"] = "Your OpenAI API Key"

# Directly using database connection details
host = "localhost"
user = "root"
password = "Your MySQL Password"
database = "SalesOrderSchema"

# Setup database connection
db_uri = f"mysql+mysqlconnector://{user}:{password}@{host}/{database}"
db = SQLDatabase.from_uri(db_uri)
llm = ChatOpenAI(model="gpt-4", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

# Streamlit app layout
st.title('SQL Chatbot')

# User input
user_query = st.text_area("Enter your SQL-related query:", "List Top 10 Employees by Salary?")

if st.button('Submit'):
    #try:
        # Processing user input
        #response = agent_executor.invoke(user_query)
        #response = agent_executor.invoke({"query": user_query})
        #if st.button('Submit'):
    try:
        # Processing user input
        response = agent_executor.invoke({
            "agent_scratchpad": "",  # Assuming this needs to be an empty string if not used
            "input": user_query  # Changed from "query" to "input"
        })
        st.write("Response:")
        st.json(response)  # Use st.json to pretty print the response if it's a JSON
    except Exception as e:
        st.error(f"An error occurred: {e}")

        #st.write("Response:")
        #st.write(response)
    #except Exception as e:
        #st.error(f"An error occurred: {e}")